In [2]:
using JSON
using DataFrames
using StatsPlots
using Turing
using LinearAlgebra
using Random


### Multi-level model using football match simulation as an example

In [3]:
## First, import the data and do some data wrangling
england_league = JSON.parsefile("./matches_England.json")

# Create a data frame with which to store the data
matches_df = DataFrame(home = [], away = [], score_home = [], score_away = [])

0×4 DataFrame
 Row │ home  away  score_home  score_away 
     │ Any   Any   Any         Any        
─────┴────────────────────────────────────

In [4]:
# example entry for each game in england_league:  "label" => "Burnley - AFC Bournemouth, 1 - 2"
# matches will exist to take each row within the england league and parse based on "," 
matches = []
for match in england_league
    push!(matches, split(match["label"], ",")) # "Burnley - AFC Bournemouth" # "1 - 2"
end

# With the parsed matches data, we can split the Home and Away Teams as well as the Home and Away Scores
for match in matches
    home, away = split(match[1], " - ")  # "Burnley" # "AFC Bournemouth" 
    score_home, score_away = split(match[2], " - ") # "1" # "2"
    push!(matches_df,[home, away, parse(Int,score_home), parse(Int,score_away)]) # push the results of parsing into the matches data frame
end

matches_df # display the matches data frame

teams = unique(collect(matches_df[:,1])) # gather the unique team names from each list

20-element Vector{Any}:
 "Burnley"
 "Crystal Palace"
 "Huddersfield Town"
 "Liverpool"
 "Manchester United"
 "Newcastle United"
 "Southampton"
 "Swansea City"
 "Tottenham Hotspur"
 "West Ham United"
 "Manchester City"
 "Leicester City"
 "Chelsea"
 "Arsenal"
 "Everton"
 "AFC Bournemouth"
 "Watford"
 "West Bromwich Albion"
 "Stoke City"
 "Brighton & Hove Albion"

In [5]:
## Now, our model

@model function football_matches(home_teams, away_teams, score_home, score_away, teams)

    # Hyper priors
    μatt ~ Normal(0, 0.1)
    μdef ~ Normal(0, 0.1)
    σatt ~ Exponential(1)
    σdef ~ Exponential(1)
    home ~ Normal(0, 1)
        
    # Team-specific effects
    att = zeros(length(teams))
    def = zeros(length(teams))
    
    # create distributions for each team given the distribution of hyper parameters (this is where we deviate from a signle level model)
    att ~ filldist(Normal(μatt, σatt), length(teams))  # more compact than for-loops
    def ~ filldist(Normal(μdef, σdef), length(teams))
    
    # offset created to remove any averaged bias among teams
    offset = mean(att) + mean(def)
    
    # the number of matches
    n_matches = length(home_teams)

    # scoring rates θ
    θ_home = Vector{Real}(undef, n_matches)     # or just θ_home = zeros(n_matches)
    θ_away = Vector{Real}(undef, n_matches)     # or just θ_away = zeros(n_matches)
        
    # Modeling score-rate and scores for each match
    for i in 1:n_matches
        # Get index of home team and away team based on the current match of the season "i"
        home_team_idx = findfirst(isequal(home_teams[i]), teams)
        away_team_idx = findfirst(isequal(away_teams[i]), teams)

        # create an array to record the distribution of scoring for each home/away game "i" based on the two teams playing
        θ_home[i] = home + att[home_team_idx] + def[away_team_idx] - offset
        θ_away[i] = att[away_team_idx] + def[home_team_idx] - offset

        # predict the score of each game here via a poisson distribution, exponential added to ensure positivity for scores
        score_home[i] ~ Poisson(exp(θ_home[i]))  # To ensure positive λ for Poisson distribution
        score_away[i] ~ Poisson(exp(θ_away[i]))
    end
end

football_matches (generic function with 2 methods)

In [6]:
# Create the model based on: ([home team, away team, home score, away score], league teams)
model = football_matches(matches_df[:,1], matches_df[:,2], matches_df[:,3], matches_df[:,4], teams);

# Markov Chain Monte Carlo Prediction to get posterior distribution of scores for each match 
# Ex: Liverpool - Arsenal Scores: (2, 3), (3,2), (1,1), etc..... and games can be either home or away for either team
posterior = sample(model, NUTS(), 3000) 

┌ Info: Found initial step size
└   ϵ = 0.2
Sampling: 100%|█████████████████████████████████████████| Time: 0:06:02


Chains MCMC chain (3000×57×1 Array{Float64, 3}):

Iterations        = 1001:1:4000
Number of chains  = 1
Samples per chain = 3000
Wall duration     = 393.38 seconds
Compute duration  = 393.38 seconds
parameters        = μatt, μdef, σatt, σdef, home, att[1], att[2], att[3], att[4], att[5], att[6], att[7], att[8], att[9], att[10], att[11], att[12], att[13], att[14], att[15], att[16], att[17], att[18], att[19], att[20], def[1], def[2], def[3], def[4], def[5], def[6], def[7], def[8], def[9], def[10], def[11], def[12], def[13], def[14], def[15], def[16], def[17], def[18], def[19], def[20]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        μatt    0.0036 

In [7]:
posterior_df=DataFrame(posterior) # Create a data-frame of the posterior

3000×59 DataFrame
  Row │ iteration  chain  μatt        μdef         σatt      σdef      home    ⋯
      │ Int64      Int64  Float64     Float64      Float64   Float64   Float64 ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │      1001      1   0.318117   -0.0219766   0.473065  0.153446  0.26115 ⋯
    2 │      1002      1   0.204412   -0.0956549   0.370176  0.16197   0.33925
    3 │      1003      1   0.177338   -0.170037    0.569271  0.197696  0.34528
    4 │      1004      1   0.202314   -0.142849    0.297959  0.196513  0.39365
    5 │      1005      1   0.180585   -0.0538004   0.41099   0.173685  0.38267 ⋯
    6 │      1006      1   0.0291641  -0.113368    0.384927  0.195031  0.31512
    7 │      1007      1   0.0259697  -0.196606    0.520989  0.236092  0.27693
    8 │      1008      1  -0.148033   -0.0675587   0.416358  0.266034  0.39625
    9 │      1009      1   0.038898    0.0743044   0.377778  0.179079  0.26529 ⋯
   10 │      1010      1  -0.070417   -0.0099039   0.438076  0.216729  0.35523
   11 │      1011      1   0.0273687  -0.0893843   0.363286  0.327957  0.24670
  ⋮   │     ⋮        ⋮        ⋮            ⋮          ⋮         ⋮         ⋮    ⋱
 2991 │      3991      1   0.138449    0.0771349   0.422261  0.205469  0.38630
 2992 │      3992      1  -0.118862    0.0245494   0.310225  0.240305  0.31691 ⋯
 2993 │      3993      1   0.0669985   0.0200462   0.357817  0.257071  0.37057
 2994 │      3994      1  -0.0946729   0.0704275   0.332389  0.162098  0.31297
 2995 │      3995      1  -0.0547062   0.0203816   0.460221  0.132667  0.40043
 2996 │      3996      1  -0.199673    0.077947    0.372886  0.2715    0.28275 ⋯
 2997 │      3997      1   0.0263468  -0.0540516   0.484705  0.218525  0.35925
 2998 │      3998      1  -0.0553182   0.116931    0.434292  0.242294  0.32112
 2999 │      3999      1  -0.177896    0.153853    0.312816  0.16507   0.38191
 3000 │      4000      1  -0.0886406   0.0472797   0.348221  0.177146  0.36874 ⋯
                                                53 columns and 2979 rows omitted

In [8]:
# adding columns to the Data Frame which will include attacking power mean, defensive power mean, and offset sum
# this is calculated using the ByRow function indicating the direction with the statistical property for an argument
DataFrames.transform!(posterior_df, AsTable(Between("att[1]","att[20]")) => ByRow(mean) => :att_mean)
DataFrames.transform!(posterior_df, AsTable(Between("def[1]","def[20]")) => ByRow(mean) => :def_mean)
DataFrames.transform!(posterior_df, AsTable([:att_mean,:def_mean]) => ByRow(sum) => :offset)

3000×62 DataFrame
  Row │ iteration  chain  μatt        μdef         σatt      σdef      home    ⋯
      │ Int64      Int64  Float64     Float64      Float64   Float64   Float64 ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │      1001      1   0.318117   -0.0219766   0.473065  0.153446  0.26115 ⋯
    2 │      1002      1   0.204412   -0.0956549   0.370176  0.16197   0.33925
    3 │      1003      1   0.177338   -0.170037    0.569271  0.197696  0.34528
    4 │      1004      1   0.202314   -0.142849    0.297959  0.196513  0.39365
    5 │      1005      1   0.180585   -0.0538004   0.41099   0.173685  0.38267 ⋯
    6 │      1006      1   0.0291641  -0.113368    0.384927  0.195031  0.31512
    7 │      1007      1   0.0259697  -0.196606    0.520989  0.236092  0.27693
    8 │      1008      1  -0.148033   -0.0675587   0.416358  0.266034  0.39625
    9 │      1009      1   0.038898    0.0743044   0.377778  0.179079  0.26529 ⋯
   10 │      1010      1  -0.070417   -0.0099039   0.438076  0.216729  0.35523
   11 │      1011      1   0.0273687  -0.0893843   0.363286  0.327957  0.24670
  ⋮   │     ⋮        ⋮        ⋮            ⋮          ⋮         ⋮         ⋮    ⋱
 2991 │      3991      1   0.138449    0.0771349   0.422261  0.205469  0.38630
 2992 │      3992      1  -0.118862    0.0245494   0.310225  0.240305  0.31691 ⋯
 2993 │      3993      1   0.0669985   0.0200462   0.357817  0.257071  0.37057
 2994 │      3994      1  -0.0946729   0.0704275   0.332389  0.162098  0.31297
 2995 │      3995      1  -0.0547062   0.0203816   0.460221  0.132667  0.40043
 2996 │      3996      1  -0.199673    0.077947    0.372886  0.2715    0.28275 ⋯
 2997 │      3997      1   0.0263468  -0.0540516   0.484705  0.218525  0.35925
 2998 │      3998      1  -0.0553182   0.116931    0.434292  0.242294  0.32112
 2999 │      3999      1  -0.177896    0.153853    0.312816  0.16507   0.38191
 3000 │      4000      1  -0.0886406   0.0472797   0.348221  0.177146  0.36874 ⋯
                                                56 columns and 2979 rows omitted

In [9]:
# For this example, we are interested in a pair of teams (no need to use map for all teams)

# Changed this up because I am a Liverpool Fan and have too many of my friends are Chelsea fans
# Hopefully you are not a die hard Chelsea fan :D
teamA = "Liverpool"
teamB = "Chelsea"

# finding the index of Liverpool and Chelsea within the 'teams' list
teamA_id = findfirst(isequal(teamA), teams)
teamB_id = findfirst(isequal(teamB), teams)

# with the associated index, find their posterior attack and defensive powers
teamA_att_post = posterior_df[:,"att[$teamA_id]"]
teamA_def_post = posterior_df[:,"def[$teamA_id]"]

teamB_att_post = posterior_df[:,"att[$teamB_id]"]
teamB_def_post = posterior_df[:,"def[$teamB_id]"]

3000-element Vector{Float64}:
 -0.06015468035318201
 -0.26122755272163406
 -0.26321696350929663
 -0.32692392345447907
 -0.27269584887193743
 -0.14236587113035684
 -0.45868731005446467
 -0.27514216392937463
 -0.1401638893979893
 -0.2698934645552743
 -0.2248664606066604
 -0.3787016913883121
 -0.44537635661071795
  ⋮
  0.010370058336212296
 -0.33584283884511756
  0.18120280647905604
 -0.32970626168438144
  0.06035548915652385
 -0.2853752421488858
  0.07294447467466675
 -0.37505125139991896
  0.08221411363958459
 -0.15855978512986152
 -0.02542182766658673
 -0.10420145056510563

In [57]:
# histogram plotting for the attack power of each team
ha1 = histogram(teamA_att_post, title=teamA*" attack", titlefontsize = 12, legend=false, normalized=true, color=:red);
ha2 = histogram(teamB_att_post, title=teamB*" attack", titlefontsize = 12, legend=false, normalized=true, color=:blue);
ha = plot(ha1, ha2, layout=(1,2));
xlabel!("Attack power");
ylabel!("Probability density")
savefig(ha,"Attack Power.png")

"c:\\Users\\dylan\\Documents\\MATLAB\\UCLA\\M298\\MP2\\Attack Power.png"

In [58]:
# histogram plotting for defensive power of each team 
hd1 = histogram(teamA_def_post, title=teamA*" defense", titlefontsize = 12, legend=false, normalized=true, color=:red);
hd2 = histogram(teamB_def_post, title=teamB*" defense", titlefontsize = 12, legend=false, normalized=true, color=:blue);
hd = plot(hd1, hd2, layout=(1,2));
xlabel!("Defense power");
ylabel!("Probability density")
savefig(hd,"Defensive Power.png")

"c:\\Users\\dylan\\Documents\\MATLAB\\UCLA\\M298\\MP2\\Defensive Power.png"

### Mini Project

Consult the lecture notes.

In [ ]:

Random.seed!(1234)
# hint: let's simulate 500 hypothetical finals (then you will have a total of 3000*500 samples)

# first leg: teamA is the home team and teamB is the away team
θ_home = posterior_df[:,:home] + posterior_df[:,"att[$teamA_id]"] + posterior_df[:,"def[$teamB_id]"] - posterior_df[:,:offset]
θ_away = posterior_df[:,"def[$teamA_id]"] + posterior_df[:,"att[$teamB_id]"] - posterior_df[:,:offset]

teamA_score = rand.(Poisson.(exp.(θ_home)),500)
teamB_score = rand.(Poisson.(exp.(θ_away)),500)

# second leg: teamA is the away team and teamB is the home team
θ_home = posterior_df[:,:home] + posterior_df[:,"def[$teamA_id]"] + posterior_df[:,"att[$teamB_id]"] - posterior_df[:,:offset]
θ_away = posterior_df[:,"att[$teamA_id]"] + posterior_df[:,"def[$teamB_id]"] - posterior_df[:,:offset]

teamA_score += rand.(Poisson.(exp.(θ_away)),500)  # add the first-leg result with the second-leg result
teamB_score += rand.(Poisson.(exp.(θ_home)),500)

# transform into long column vectors
teamA_score = vcat(teamA_score...)
teamB_score = vcat(teamB_score...)

Arange = sort(unique(teamA_score))
Brange = sort(unique(teamB_score))

# 2D histogram for final score results
h1 = histogram2d(teamA_score, teamB_score; color =:turbo, yaxis=(tickvals=Brange), xaxis=(tickvals=Arange), title="Premier League Final - Histogram", titlefontsize = 12)
xlabel!(teamA*" Score")
ylabel!(teamB*" Score")
savefig(h1, "histogram.png")


"c:\\Users\\dylan\\Documents\\MATLAB\\UCLA\\M298\\MP2\\histogram.png"

In [ ]:
# Generate Statistics for Liverpool over 500 2-leg matches
A_avg = mean(teamA_score)
A_var = std(teamA_score)
Amin = findmin(teamA_score)
Amax = findmax(teamA_score)
Apdf = range(Amin[1],Amax[1], length = 250)

# Generate Statistics for Chelsea over 500 2-leg matches
B_avg = mean(teamB_score)
B_var = std(teamB_score)
Bmin = findmin(teamB_score)
Bmax = findmax(teamB_score)
Bpdf = range(Bmin[1],Bmax[1], length = 250)

# Generate PDF assuming Gaussian Distribution over 500 2-leg matches
pdfAB = zeros(length(Apdf),length(Bpdf))
for i = 1:length(Apdf)
    for j = 1:length(Bpdf)
        pdfAB[i,j] = 1/(sqrt(2*pi)*A_var)*exp(-(Apdf[i]-A_avg)^2/(2*A_var^2))* 1/(sqrt(2*pi)*B_var)*exp(-(Bpdf[j]-B_avg)^2/(2*B_var^2))
    end
end

# Heatmap of the PDF 
h2 = heatmap(Apdf,Bpdf,pdfAB; color=:turbo, colorbar=false, yaxis=(tickvals=Brange), xaxis=(tickvals=Arange), title="Premier League 500 Match Final - Heatmap", titlefontsize = 12)
xlabel!(teamA*" Score")
ylabel!(teamB*" Score")
savefig(h2, "heatmap.png")

"c:\\Users\\dylan\\Documents\\MATLAB\\UCLA\\M298\\MP2\\heatmap.png"